# LDA on Independent Years

As we were unable to get the LDA sequence model to work efficiently, we decided to see what the effect would be if we looked at the years independently and then compare this to the LDA across the whole dataset. 

In [1]:
import gensim
import pandas as pd
from ast import literal_eval
from gensim.models.coherencemodel import CoherenceModel
from sklearn.model_selection import KFold
import statistics

In [2]:
df = pd.read_csv("../data/processed/formatted_df.csv").drop(columns = ['Unnamed: 0'])
df

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Name,Status,Description,References,Phase,Votes,Comments
0,CVE-1999-0001,Candidate,"['ipinputc', 'bsdderived', 'tcpip', 'implement...",BUGTRAQ:19981223 Re: CERT Advisory CA-98.13 - ...,Modified (20051217),"MODIFY(1) Frech | NOOP(2) Northcutt, W...",Christey> A Bugtraq posting indicates that the...
1,CVE-1999-0002,Entry,"['buffer', 'overflow', 'nfs', 'mountd', 'give'...",BID:121 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
2,CVE-1999-0003,Entry,"['execute', 'command', 'root', 'buffer', 'over...",BID:122 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
3,CVE-1999-0004,Candidate,"['mime', 'buffer', 'overflow', 'email', 'clien...",CERT:CA-98.10.mime_buffer_overflows | MS:M...,Modified (19990621),"ACCEPT(8) Baker, Cole, Collins, Dik, Landfi...","Frech> Extremely minor, but I believe e-mail i..."
4,CVE-1999-0005,Entry,"['arbitrary', 'command', 'execution', 'imap', ...",BID:130 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
166896,CVE-2021-46482,Candidate,"['jsish', 'v', 'discover', 'contain', 'heap', ...",MISC:https://github.com/pcmacdon/jsish/issues/66,Assigned (20220124),None (candidate not yet proposed),NaN
166897,CVE-2021-46483,Candidate,"['jsish', 'v', 'discover', 'contain', 'heap', ...",MISC:https://github.com/pcmacdon/jsish/issues/62,Assigned (20220124),None (candidate not yet proposed),NaN
166898,CVE-2021-46559,Candidate,"['firmware', 'moxa', 'tn', 'device', 'weak', '...",MISC:https://www.moxa.com/en/support/product-s...,Assigned (20220126),None (candidate not yet proposed),NaN
166899,CVE-2021-46560,Candidate,"['firmware', 'moxa', 'tn', 'device', 'allow', ...",MISC:https://www.moxa.com/en/support/product-s...,Assigned (20220126),None (candidate not yet proposed),NaN


We found that the way that we saved the data frame, meant that the Description column was read as a string rather than a list as it was intended. Therefore, we had to apply the function literal_eval which allows us to convert the string of a stored list into a python list. We then separate the description column into a list to allow easier access. 

In [3]:
df['Description'] = df['Description'].apply(literal_eval)

In [4]:
desc = df['Description']

In [5]:
names = df['Name']
year = []
for instance in names:
    year.append(int(instance[4:8]))
year_count = [0]
for i in range(23):
    if i == 0:
        year_count.append(year.count(i+1999))
    else:
        year_count.append(year.count(i+1999) + year_count[i]) 
print(year_count)

[0, 1541, 2778, 4313, 6663, 8161, 10794, 15380, 22238, 28578, 35549, 40436, 45428, 50015, 55416, 61536, 69815, 77731, 86931, 101250, 116731, 132000, 149784, 166901]


Here we create a dictionary of words that occur in the whole data set, allowing us to index each of these words. We also format the documents of each year into a matrix which indicates how many times each word occurs in each document. 

In [6]:
vocab = gensim.corpora.Dictionary(desc)
doc_word_matrix_array = []
for i in range(23):
    doc_word_matrix_array.append([vocab.doc2bow(doc) for doc in desc[year_count[i]:year_count[i+1]]]) 

In [7]:
LDA = gensim.models.ldamodel.LdaModel

### Model Training using CV

After trying to find an optimal number of topics, we decided to use 50 as this is what was used in Analyzing Evolving Trends of Vulnerabilities in National Vulnerability Database by Williams et al., which is what provided us the idea for this project. We did a 6-Fold cross validation on each year as this helps to ensure that we don't overfit and so helps provide a better overview of how the topics change over time. 

In [13]:
kf = KFold(n_splits=6, random_state=27, shuffle=True)
ldamodels=[]
for i in range(23):
    models = []
    coherence = []
    j=0
    for split in kf.split(desc[year_count[i]:year_count[i+1]]): 
        train = [vocab.doc2bow(doc) for doc in desc[split[0]]]
        test = [vocab.doc2bow(doc) for doc in desc[split[1]]]
        models.append(LDA(corpus=train, id2word=vocab, passes = 3, num_topics = 50))  
        coherence.append(CoherenceModel(model=models[j], corpus=test, dictionary=vocab, coherence='u_mass').get_coherence())
        j += j
    ldamodels.append(models[coherence.index(min(coherence))]) #append the model, corresponding to the best coherence, to the list of final yearly models

/home/c/.local/lib/python3.8/site-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words


The below was created for Bill to use on his model.

In [ ]:
#THIS IS TO BE USED FOR THE CV FOR LDA ON THE WHOLE DATA SET

vocab = gensim.corpora.Dictionary(desc)
doc_word_matrix_array = []
for i in range(23):
    doc_word_matrix_array.append([vocab.doc2bow(doc) for doc in desc[year_count[i]:year_count[i+1]]]) 


kf = KFold(n_splits=6, random_state=27, shuffle=True)
train=[[] * 6] 
test=[[] * 6]
for i in range(23): # For each year work out the CV split and combine all of the years into one big test and train dataset
    j=0
    for split in kf.split(desc[year_count[i]:year_count[i+1]]):
        train[j].extend([vocab.doc2bow(doc) for doc in desc[split[0]]])
        test[j].extend([vocab.doc2bow(doc) for doc in desc[split[1]]])
        j += j      
for j in range(6):
    temp_model = LDA(corpus=train[j], id2word=vocab, passes = 3, num_topics = 50) #Traing model here 
    temp_coherence = CoherenceModel(model=model, corpus=test[j], dictionary=vocab, coherence='u_mass').get_coherence()
    if temp_coherence > coherence:
        model = temp_model
        coherence = temp_coherence

### Topic matching between years

Tried this implementation but did not work, accoriding to an issue reported on the Github, this is because there are too many nodes, so I will use a different implementation of the Hungarian algorithm.

In [ ]:
#!pip install hungarian-algorithm
from hungarian-algorithm import algorithm

In [ ]:
#used to extract the top 50 words in each topic and return a list of topics containing the list of 50 words
def extract_Words(model): 
    topics=[]
    topic_index=[]
    for words in model.show_topics(num_topics = 50, num_words = 50, formatted=False):
        (a,b) = words #removes the index
        word=[]
        for j in b:
            (c,d) = j #extracts the word from the (word, probability) tuple
            word.append(c)
        topics.append(word)
        topic_index.append(int(a))
    return (topics,topic_index)

#Creates a dictionary of jaccard scores
#Requires topics1 and topics2 to have 50 topics in each
def graphify(topics1, topics2):
    G = {}
    for i in range(50):
        jaccard_dict = {}
        for j in range(50):
            jaccard_dict[j+50] = jaccard_score(topics1[i],topics2[j],average='macro')
        G[i] = jaccard_dict
        print(jaccard_dict)
    return G

#find best match between topic-sets and change topic-set 2 numbering to replicate topic-set 1.
def match_sets(G,index1):
    matching = algorithm.find_matching(G, matching_type = 'max', return_type = 'list')
    new_index2=[]*50
    for i in range(50):
        (a,b) = matching[i]
        (c,d) = a
        new_index2[d-50]=index1[c] 
    return new_index2

topicWords=[]
indices=[]
for i in range(23):
    (topics, indexes) = extract_Words(ldamodels[i])
    topicWords.append(topics)
    indices.append(indexes)
topic_map = [Indices[0]]
for i in range(22):
    G = graphify(topicWords[i],topicWords[i+1])
    next_topic_index = match_sets(G,indices[i])
    topic_map.append(next_topic_index)

We now need to correlate the topics from each year to eachother. This is important for our analysis as the topic labelled 27 in year 1999 could be related to DDoS vulnerabilities, while this could be labelled as 9 in year 2000. Therefore, we need to correlate all the topics so we can see how the proportions of the same topic changes. To do this we are going to use a Jaccard similarity to work out how similar a topic from year i compares to each of the topics in year i+1. We then create a matrix of this and solve the matching problem that maximises the similarity of the topics between years. To do this we use the linear_sum_assignment function to solve this efficiently. 

In [ ]:
#used to extract the top 500 words in each topic and return a list of topics containing the list of 500 words
def extract_Words(model): 
    topics=[]
    topic_index=[]
    for words in model.show_topics(num_topics = 50, num_words = 500, formatted=False):
        (a,b) = words #removes the index
        word=[]
        for j in b:
            (c,d) = j #extracts the word from the (word, probability) tuple
            word.append(c)
        topics.append(word)
        topic_index.append(int(a))
    return (topics,topic_index)

#Creates a matrix of the Jaccard scores
#Requires topics1 and topics2 to have 50 topics in each
def calculate_cost(topics1, topics2):
    cost = []
    for i in range(50):
        jaccard = []
        for j in range(50):
            jaccard.append(jaccard_score(topics1[i],topics2[j],average='macro'))
        cost.append(jaccard) 
    return cost

#find best match between topic-sets and change topic-set 2 numbering to replicate topic-set 1.
def match_sets(cost,index1):
    row_ind, col_ind = linear_sum_assignment(cost)
    new_index2=[0]*50
    for i in range(50):
        new_index2[col_ind[i]]=index1[row_ind[i]] 
    return new_index2

topicWords=[]
indices=[]
for i in range(23):
    (topics, indexes) = extract_Words(ldamodels[i])
    topicWords.append(topics)
    indices.append(indexes)
topic_map = [Indices[0]]
for i in range(22):
    cost = calculate_cost(topicWords[i],topicWords[i+1])
    if i == 0:
        print(cost[0])
    next_topic_index = match_sets(cost,indices[i])
    topic_map.append(next_topic_index)

Originally, we looked at the top 50 words from each topic, however we found that the jaccard scores were very low and many were 0. Therefore, we decided to increase the number of words to 500 as we thought that this would provide enough words to compare, while allowing us to compute in a reasonable time. As you can see from the output (which is the Jaccard scores for topic\[0\] in year 1999 compared to each of the topics in year 2000), this did not have as much of an effect as we had hoped. It is clear from the output that there is very little correlation between topics generated from the LDA model in each year. It is possible that it is showing that there is a very great change between the different vulnerabilties reported in each year, however we believe it is much more likely that it is a poor model that we have created. Therefore, we have decided that this approach does not provide a useful result that we can use to analyse the change of vulnerability types over time. 

It would have been nice to have come up with a similarity measure that took into consideration the probabilities of each word occuring in a topic, however we did not have time to implement this. 